In [ ]:
import pandas as pd
import numpy as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Создаем директорию для сохранения моделей, если она не существует
os.makedirs('models', exist_ok=True)

# Загрузка данных
df = pd.read_csv('Data/combined_features.csv')

# Обработка бесконечных и NaN значений
df = df.replace([np.inf, -np.inf], np.nan)

# Заполнение NaN значений для каждого числового столбца
for column in df.select_dtypes(include=[np.number]).columns:
    # Если все значения в столбце - NaN, заполняем стандартным значением
    if df[column].isna().all():
        df[column] = 1000000  # Используем большое число как заполнитель
    else:
        # Иначе заполняем NaN значения удвоенным максимальным значением столбца
        max_val = df[column].max()
        df[column] = df[column].fillna(max_val * 2)

# Обработка MAC-адресов (преобразование в числовой формат)
mac_columns = [col for col in df.columns if 'mac' in col.lower()]
for col in mac_columns:
    # Удаляем двоеточия и дефисы из MAC-адресов
    df[f"{col}_int"] = df[col].str.replace(':', '').str.replace('-', '')
    # Преобразуем в целочисленный тип
    df[f"{col}_int"] = df[f"{col}_int"].apply(lambda x: int(x, 16) if pd.notna(x) and x != '' else 0)
    # Удаляем исходный столбец
    df = df.drop(columns=[col])

# Обработка IP-адресов (преобразование в числовой формат)
ip_columns = [col for col in df.columns if 'ip' in col.lower()]
for col in ip_columns:
    # Функция для преобразования IP в число
    def ip_to_int(ip):
        try:
            if pd.isna(ip) or ip == '':
                return 0
            return sum(int(octet) * (256 ** (3 - i)) for i, octet in enumerate(ip.split('.')))
        except:
            return 0
    
    df[f"{col}_int"] = df[col].apply(ip_to_int)
    # Удаляем исходный столбец
    df = df.drop(columns=[col])

# Обработка булевых и категориальных признаков
bool_columns = df.select_dtypes(include=['bool']).columns
for col in bool_columns:
    df[col] = df[col].astype(int)

# Проверка и обработка оставшихся нечисловых столбцов
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"Преобразование столбца {col} в числовой формат")
        # Используем one-hot encoding для категориальных признаков
        df = pd.get_dummies(df, columns=[col], drop_first=True)

# Обработка оставшихся NaN-значений (заполнение средними значениями)
df = df.fillna(df.mean())

# Разделение на признаки и целевую переменную
X = df.drop('is_attack', axis=1)
y = df['is_attack']

# Масштабирование числовых признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение на обучающую, валидационную и тестовую выборки
X_train_val, X_test, y_train_val, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер валидационной выборки: {X_val.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")

# Создание модели
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(64, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(32, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Коллбэки для обучения
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='models/arp_spoofing_detector.keras',
        monitor='val_loss',
        save_best_only=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.00001
    )
]

# Обучение модели
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# Оценка модели на тестовой выборке
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).reshape(-1)

# Расчет метрик качества
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Точность (Accuracy): {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")

# Сохранение метрик в JSON
metrics = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1_score": f1,
    "confusion_matrix": conf_matrix.tolist()
}

with open('models/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

# Сохранение модели
model.save('models/arp_spoofing_detector.keras')

# Сохранение scaler
with open('models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Построение графиков обучения
plt.figure(figsize=(15, 5))

# График точности
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Обучающая выборка')
plt.plot(history.history['val_accuracy'], label='Валидационная выборка')
plt.title('Точность модели')
plt.ylabel('Точность')
plt.xlabel('Эпоха')
plt.legend()

# График функции потерь
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Обучающая выборка')
plt.plot(history.history['val_loss'], label='Валидационная выборка')
plt.title('Функция потерь')
plt.ylabel('Потери')
plt.xlabel('Эпоха')
plt.legend()

plt.tight_layout()
plt.savefig('models/training_history.png')

# Построение матрицы ошибок
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Normal', 'Attack'],
            yticklabels=['Normal', 'Attack'])
plt.title('Матрица ошибок')
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.tight_layout()
plt.savefig('models/confusion_matrix.png')

print("Обучение завершено. Модель и результаты сохранены в директории 'models/'")